# EDA — Estação Tijuca (Dados Horários de Qualidade do Ar)

Este notebook realiza uma **análise exploratória completa** (EDA) dos dados horários da **Estação Tijuca**,
com foco nas variáveis meteorológicas e de poluentes atmosféricos. O fluxo foi desenhado para ser **reutilizável**
em outras estações, bastando trocar o caminho do arquivo de entrada.

## O que este notebook faz
1. **Carregamento e inspeção** do conjunto de dados (tipos, dimensões, faltantes, duplicados).
2. **Tratamento de datas** e criação de colunas temporais (dia, mês, ano).
3. **Regra de imputação condicional** (baseada em boas práticas WMO/WHO):
   - Se **≤4 horas faltantes dentro do mesmo dia** → interpolar por dia (por variável).
   - Se **>4 horas** → manter **NaN** (evita viés em lacunas longas).
4. **Checagens de plausibilidade física** e marcação/remoção de outliers extremos.
5. **Estatísticas descritivas** por variável (média, mediana, quantis, etc.).
6. **Sazonalidade e tendência** (médias mensais/anuais, médias móveis).
7. **Distribuições** (histogramas e boxplots).
8. **Correlação** entre variáveis.
9. **Exporta** dados limpos e resumos (CSV) + **figuras** em pasta própria da estação.

> **Base científica para a regra de imputação**  
> - **WMO (2021)** e **WHO (2021)** recomendam evitar interpolação em **lacunas longas**, usando-a apenas para **pequenos hiatos** (algumas horas).  
> - Estudos (e.g., Vicedo‑Cabrera et al., *Environmental Research*, 2018) mostram que imputações inadequadas podem **subestimar picos** e **enviesar análises** associadas a saúde.

> **Entrada esperada**: um CSV com colunas como `nome_estacao`, `data`, `chuva`, `temp`, `ur`, `co`, `no`, `no2`, `nox`, `so2`, `o3`, `pm10`, `pm2_5`, `lat`, `lon`.


## Configurações e importações
Bibliotecas usadas e parâmetros gerais.

In [1]:
import os
import math
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from pathlib import Path as _Path

# Pastas de saída
ESTACAO = "TIJUCA"
OUT_BASE = Path(f"resultados_estacao_{ESTACAO}")
FIG_DIR = OUT_BASE / "figuras"
DATA_DIR = OUT_BASE / "dados"
OUT_BASE.mkdir(exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = _Path(f"resultados_estacao_{ESTACAO}") / "dados"
DATA_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR = _Path(f"resultados_estacao_{ESTACAO}") / "figuras"
FIG_DIR.mkdir(parents=True, exist_ok=True)

INPUT_PATH = 'https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataRio/Estacoes/ESTACAO_TIJUCA.csv'

print("Usando arquivo:", INPUT_PATH)


Usando arquivo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataRio/Estacoes/ESTACAO_TIJUCA.csv


In [2]:
# Helper para criar uma pasta por coluna 
def _col_dir(base_dir, col_name):
    safe = str(col_name).strip().lower().replace(" ", "_").replace("/", "_").replace("\\", "_")
    d = base_dir / safe
    d.mkdir(parents=True, exist_ok=True)
    return d

## Carregamento e inspeção inicial
Verificamos dimensões, tipos, amostras, faltantes e duplicados.

In [3]:
df = pd.read_csv(INPUT_PATH)

print("Dimensões:", df.shape)
print("\nTipos:")
print(df.dtypes)

print("\nAmostra:")
display(df.head())

print("\nValores ausentes por coluna:")
print(df.isna().sum())

# Duplicados (por carimbo horário e nome_estacao, se houver)
dup_cols = [c for c in ['nome_estacao', 'data'] if c in df.columns]
if dup_cols:
    ndup = df.duplicated(subset=dup_cols).sum()
    print(f"\nRegistros duplicados por {dup_cols}: {ndup}")
else:
    print("\nColunas para checar duplicados não disponíveis (nome_estacao/data).")


Dimensões: (113976, 23)

Tipos:
nome_estacao       object
codnum              int64
data               object
chuva             float64
temp              float64
ur                float64
pres              float64
rs                float64
dir_vento         float64
vel_vento         float64
co                float64
no                float64
no2               float64
nox               float64
so2               float64
o3                float64
pm10              float64
pm2_5             float64
lat               float64
lon               float64
data_formatada     object
ano                 int64
mes                 int64
dtype: object

Amostra:


,nome_estacao,codnum,data,chuva,temp,ur,pres,rs,dir_vento,vel_vento,...,nox,so2,o3,pm10,pm2_5,lat,lon,data_formatada,ano,mes
0,ESTAÇÃO TIJUCA,8,2012-01-01 00:30:00,0.0,24.62,88.31,1007.32,12.88,216.50,2.28,...,NaN,0.36,24.71,27.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1
1,ESTAÇÃO TIJUCA,8,2012-01-01 01:30:00,0.4,23.50,93.05,1008.05,11.35,217.00,1.65,...,NaN,0.28,22.74,28.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1
2,ESTAÇÃO TIJUCA,8,2012-01-01 02:30:00,1.0,22.51,98.06,1008.63,11.20,197.83,0.58,...,NaN,0.27,19.22,28.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1
3,ESTAÇÃO TIJUCA,8,2012-01-01 03:30:00,0.0,21.47,98.40,1007.10,11.15,194.33,0.00,...,NaN,1.51,24.55,36.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1
4,ESTAÇÃO TIJUCA,8,2012-01-01 04:30:00,0.2,21.36,98.41,1006.13,11.08,193.33,0.00,...,NaN,1.89,19.00,40.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1



Valores ausentes por coluna:
nome_estacao           0
codnum                 0
data                   0
chuva               5835
temp                6992
ur                  7138
pres                4541
rs                  9071
dir_vento          17696
vel_vento          17698
co                 30987
no                 35940
no2                35805
nox                35784
so2                37665
o3                 10236
pm10                6012
pm2_5             113976
lat                    0
lon                    0
data_formatada         0
ano                    0
mes                    0
dtype: int64

Registros duplicados por ['nome_estacao', 'data']: 0


## Datas e colunas temporais
Converte `data` para `datetime` e cria `dia`, `mes`, `ano` e `data_dia`.

In [4]:
# Converte para datetime (ajuste o 'format' se necessário)
# Formatos comuns: '%Y-%m-%d %H:%M:%S', '%m/%d/%Y %I:%M:%S %p'
df['data'] = pd.to_datetime(df['data'], errors='coerce', infer_datetime_format=True)

# Colunas temporais
df['ano'] = df['data'].dt.year
df['mes'] = df['data'].dt.month
df['dia'] = df['data'].dt.day
df['data_dia'] = df['data'].dt.date

# Ordena cronologicamente
df = df.sort_values('data').reset_index(drop=True)

display(df.head())

C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\3424442276.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['data'] = pd.to_datetime(df['data'], errors='coerce', infer_datetime_format=True)


,nome_estacao,codnum,data,chuva,temp,ur,pres,rs,dir_vento,vel_vento,...,o3,pm10,pm2_5,lat,lon,data_formatada,ano,mes,dia,data_dia
0,ESTAÇÃO TIJUCA,8,2012-01-01 00:30:00,0.0,24.62,88.31,1007.32,12.88,216.50,2.28,...,24.71,27.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1,1,2012-01-01
1,ESTAÇÃO TIJUCA,8,2012-01-01 01:30:00,0.4,23.50,93.05,1008.05,11.35,217.00,1.65,...,22.74,28.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1,1,2012-01-01
2,ESTAÇÃO TIJUCA,8,2012-01-01 02:30:00,1.0,22.51,98.06,1008.63,11.20,197.83,0.58,...,19.22,28.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1,1,2012-01-01
3,ESTAÇÃO TIJUCA,8,2012-01-01 03:30:00,0.0,21.47,98.40,1007.10,11.15,194.33,0.00,...,24.55,36.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1,1,2012-01-01
4,ESTAÇÃO TIJUCA,8,2012-01-01 04:30:00,0.2,21.36,98.41,1006.13,11.08,193.33,0.00,...,19.00,40.0,NaN,-22.924915,-43.232657,2012-01-01,2012,1,1,2012-01-01


## Seleção de colunas e tipos
Mantemos apenas as colunas de interesse e garantimos tipos numéricos.

In [5]:
colunas_relevantes = ['nome_estacao', 'codnum', 'data', 'ano', 'mes', 'dia', 'data_dia', 'chuva', 'temp', 'ur', 'co', 'no', 'no2', 'nox', 'so2', 'o3', 'pm10', 'pm2_5', 'lat', 'lon']
presentes = [c for c in colunas_relevantes if c in df.columns]
df = df[presentes].copy()

# Força numérico para variáveis ambientais (ignora erros -> NaN)
num_cols = [c for c in ['chuva','temp','ur','co','no','no2','nox','so2','o3','pm10','pm2_5','lat','lon'] if c in df.columns]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

print("Colunas mantidas:", df.columns.tolist())

display(df.head())


Colunas mantidas: ['nome_estacao', 'codnum', 'data', 'ano', 'mes', 'dia', 'data_dia', 'chuva', 'temp', 'ur', 'co', 'no', 'no2', 'nox', 'so2', 'o3', 'pm10', 'pm2_5', 'lat', 'lon']


,nome_estacao,codnum,data,ano,mes,dia,data_dia,chuva,temp,ur,co,no,no2,nox,so2,o3,pm10,pm2_5,lat,lon
0,ESTAÇÃO TIJUCA,8,2012-01-01 00:30:00,2012,1,1,2012-01-01,0.0,24.62,88.31,0.07,NaN,NaN,NaN,0.36,24.71,27.0,NaN,-22.924915,-43.232657
1,ESTAÇÃO TIJUCA,8,2012-01-01 01:30:00,2012,1,1,2012-01-01,0.4,23.50,93.05,0.11,NaN,NaN,NaN,0.28,22.74,28.0,NaN,-22.924915,-43.232657
2,ESTAÇÃO TIJUCA,8,2012-01-01 02:30:00,2012,1,1,2012-01-01,1.0,22.51,98.06,0.10,NaN,NaN,NaN,0.27,19.22,28.0,NaN,-22.924915,-43.232657
3,ESTAÇÃO TIJUCA,8,2012-01-01 03:30:00,2012,1,1,2012-01-01,0.0,21.47,98.40,0.06,NaN,NaN,NaN,1.51,24.55,36.0,NaN,-22.924915,-43.232657
4,ESTAÇÃO TIJUCA,8,2012-01-01 04:30:00,2012,1,1,2012-01-01,0.2,21.36,98.41,0.08,NaN,NaN,NaN,1.89,19.00,40.0,NaN,-22.924915,-43.232657


## Imputação condicional por dia (≤6 horas)  
**Por quê?** Para evitar viés, interpolamos **apenas lacunas curtas** (≤6 horas) dentro de cada dia e variável. 

In [6]:
cols_to_interp = [c for c in ['chuva','temp','ur','co','no','no2','nox','so2','o3','pm10','pm2_5'] if c in df.columns]

### Distribuição de valores vazios

In [7]:
def plot_missing_distribution(df_, col, outdir):
    if col not in df_.columns:
        return

    if 'data_dia' not in df_.columns:
        df_ = df_.copy()
        df_['data'] = pd.to_datetime(df_['data'], errors='coerce', infer_datetime_format=True)
        df_['data_dia'] = df_['data'].dt.date

    # Contagem de nulos por dia
    daily_nans = df_.groupby('data_dia')[col].apply(lambda s: s.isna().sum()).astype(int)

    # Histograma excluindo dias sem nulos 
    x = daily_nans[daily_nans > 0]
    if not x.empty:
        plt.figure(figsize=(8,4))
        plt.hist(x, bins=range(1, int(x.max()) + 2), align='left', edgecolor='black')
        plt.title(f'Distribuição de nulos por dia — {col} (dias com > 0 nulos)')
        plt.xlabel('Nulos no dia (1–24)')
        plt.ylabel('Número de dias')
        plt.xticks(range(1, int(x.max()) + 1))
        plt.tight_layout()
        plt.savefig(outdir / "missing_hist.png", dpi=150)
        plt.close()

In [8]:
for c in cols_to_interp:
    col_dir = _col_dir(FIG_DIR, c)
    plot_missing_distribution(df, c, col_dir)

### Interpolando

In [9]:
MAX_NULOS_DIA = 6  # até 6 horas vazias no dia pode interpolar

# Garante ordenação temporal
df = df.sort_values('data').reset_index(drop=True)

for col in cols_to_interp:
    if col not in df.columns:
        continue

    # Quantidade total de valores vazios na coluna
    total_nulos = int(df[col].isna().sum())
    print(f"\n {col}: valores vazios totais = {total_nulos}")

    # Criar coluna auxiliar com a contagem de nulos por dia (repetida em cada linha do dia)
    aux_name = f"{col}_nulos_no_dia"
    df[aux_name] = (
        df[col].isna()
          .groupby(df['data_dia'])
          .transform('sum')
    )

    # Aplicar por dia apenas quando nulos_dia <= MAX_NULOS_DIA
    def _interp_if_short_gaps(g):
        missing = int(g[col].isna().sum())
        if 0 < missing <= MAX_NULOS_DIA:
            s = (
                g.set_index('data')[col]
                 .interpolate(method='time', limit_direction='both')
            )
            g[col] = s.values
        return g

    before_impute_nulls = int(df[col].isna().sum())
    df = (
        df.groupby('data_dia', group_keys=False)
          .apply(_interp_if_short_gaps)
          .reset_index(drop=True)
    )
    after_impute_nulls = int(df[col].isna().sum())
    filled = before_impute_nulls - after_impute_nulls

    perc = (100.0 * filled / total_nulos) if total_nulos > 0 else 0.0
    print(f"   - {filled} valores imputados em '{col}' (≈ {perc:.1f}% dos nulos iniciais).")

    # Remover a coluna auxiliar
    df.drop(columns=[aux_name], inplace=True)



 chuva: valores vazios totais = 5835


C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\2155526490.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_interp_if_short_gaps)


   - 321 valores imputados em 'chuva' (≈ 5.5% dos nulos iniciais).

 temp: valores vazios totais = 6992


C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\2155526490.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_interp_if_short_gaps)


   - 402 valores imputados em 'temp' (≈ 5.7% dos nulos iniciais).

 ur: valores vazios totais = 7138


...

C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\2155526490.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_interp_if_short_gaps)


   - 1230 valores imputados em 'o3' (≈ 12.0% dos nulos iniciais).

 pm10: valores vazios totais = 6012


C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\2155526490.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_interp_if_short_gaps)


   - 790 valores imputados em 'pm10' (≈ 13.1% dos nulos iniciais).

 pm2_5: valores vazios totais = 113976
   - 0 valores imputados em 'pm2_5' (≈ 0.0% dos nulos iniciais).


C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\2155526490.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_interp_if_short_gaps)


## Plausibilidade física e outliers extremos  
**Por quê?** Sensores podem registrar leituras impossíveis (erros, panes).  
Definimos **limites amplos** para marcar valores absurdos como `NaN` (não capamos por padrão).

In [10]:
bounds = {
    # Meteorologia
    'temp': (-5, 55),          # °C
    'ur': (0, 100),            # %
    'chuva': (0, 200),         # mm (acum. horário; alto para extremos)

    # Poluentes — ppm
    'co': (0, 50),             # ppm

    # Poluentes — µg/m³
    'no': (0, 2000),
    'no2': (0, 1000),
    'nox': (0, 2500),
    'so2': (0, 2000),
    'o3': (0, 1000),
    'pm10': (0, 1000),
    'pm2_5': (0, 1000),
}

df_clean = df.copy()
for col, (lo, hi) in bounds.items():
    if col in df_clean.columns:
        out_before = df_clean[col].isna().sum()
        df_clean.loc[(df_clean[col] < lo) | (df_clean[col] > hi), col] = np.nan
        out_after = df_clean[col].isna().sum()
        if out_after > out_before:
            print(f"{col}: +{out_after - out_before} valores marcados como NaN por plausibilidade")
        else:
            print(f"{col}: Nenhum valor fora dos limites plausíveis")

dup_cols = [c for c in ['nome_estacao','data'] if c in df_clean.columns]
if dup_cols:
    df_clean = df_clean.drop_duplicates(subset=dup_cols)


temp: Nenhum valor fora dos limites plausíveis
ur: Nenhum valor fora dos limites plausíveis
chuva: +5 valores marcados como NaN por plausibilidade
co: Nenhum valor fora dos limites plausíveis
no: Nenhum valor fora dos limites plausíveis
no2: Nenhum valor fora dos limites plausíveis
nox: Nenhum valor fora dos limites plausíveis
so2: Nenhum valor fora dos limites plausíveis
o3: Nenhum valor fora dos limites plausíveis
pm10: Nenhum valor fora dos limites plausíveis
pm2_5: Nenhum valor fora dos limites plausíveis


## Estatísticas descritivas por coluna

In [11]:
desc = df_clean.describe(include='all').T
desc.to_csv(DATA_DIR / f"estatisticas_{ESTACAO}.csv")
display(desc)

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
nome_estacao,113976,1,ESTAÇÃO TIJUCA,113976,NaN,NaN,NaN,NaN,NaN,NaN,NaN
codnum,113976.0,NaN,NaN,NaN,8.0,8.0,8.0,8.0,8.0,8.0,0.0
data,113976,NaN,NaN,NaN,2018-07-02 12:00:00,2012-01-01 00:30:00,2015-04-02 06:15:00,2018-07-02 12:00:00,2021-10-01 17:45:00,2024-12-31 23:30:00,NaN
ano,113976.0,NaN,NaN,NaN,2018.0,2012.0,2015.0,2018.0,2021.0,2024.0,3.742349
mes,113976.0,NaN,NaN,NaN,6.522215,1.0,4.0,7.0,10.0,12.0,3.448914
dia,113976.0,NaN,NaN,NaN,15.731733,1.0,8.0,16.0,23.0,31.0,8.801016
data_dia,113976,4749,2012-01-01,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chuva,108457.0,NaN,NaN,NaN,0.144681,0.0,0.0,0.0,0.0,197.04,2.169855
temp,107386.0,NaN,NaN,NaN,24.828252,11.32,20.68,23.92,28.02,49.16,5.721383
ur,107249.0,NaN,NaN,NaN,65.009346,10.33,51.67,66.88,79.17,99.91,19.084087


## Séries temporais e médias móveis (7 e 30 dias)
Gera gráficos individuais por variável.

In [12]:
def plot_series_with_roll(df_, col, outdir):
    if col not in df_.columns:
        return
    s = df_.set_index('data')[col].sort_index()
    if s.dropna().empty:
        return
    rm7 = s.rolling('7D').mean()
    rm30 = s.rolling('30D').mean()

    plt.figure(figsize=(12,4))
    s.plot(linewidth=0.8, label=col)
    rm7.plot(linewidth=1.0, label='mm7')
    rm30.plot(linewidth=1.2, label='mm30')
    plt.title(f"{col} — Série horária e médias móveis")
    plt.xlabel("Tempo")
    plt.ylabel(col)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outdir / "serie.png", dpi=150)
    plt.close()

## Distribuições: histograma e boxplot

In [13]:
def plot_distributions(df_, col, outdir):
    if col not in df_.columns:
        return
    x = df_[col].dropna()
    if x.empty:
        return

    # Histograma
    plt.figure(figsize=(6,4))
    plt.hist(x, bins=40)
    plt.title(f"Histograma — {col}")
    plt.xlabel(col); plt.ylabel("Frequência")
    plt.tight_layout()
    plt.savefig(outdir / "hist.png", dpi=150)
    plt.close()

    # Boxplot
    plt.figure(figsize=(4,5))
    plt.boxplot(x.values, vert=True)
    plt.title(f"Boxplot — {col}")
    plt.ylabel(col)
    plt.tight_layout()
    plt.savefig(outdir / "box.png", dpi=150)
    plt.close()


## Sazonalidade: perfis mensais e anuais

In [14]:
def monthly_profile(df_, col, outdir):
    if col not in df_.columns:
        return
    tmp = df_[['mes', col]].copy()
    g = tmp.groupby('mes')[col].mean(numeric_only=True)
    if g.dropna().empty:
        return
    plt.figure(figsize=(8,3.5))
    plt.plot(g.index, g.values, marker='o')
    plt.title(f"Média mensal — {col}")
    plt.xlabel("Mês"); plt.ylabel(col)
    plt.xticks(range(1,13))
    plt.tight_layout()
    plt.savefig(outdir / "mensal.png", dpi=150)
    plt.close()

def yearly_profile(df_, col, outdir):
    if col not in df_.columns:
        return
    tmp = df_[['ano', col]].copy()
    g = tmp.groupby('ano')[col].mean(numeric_only=True)
    if g.dropna().empty:
        return
    plt.figure(figsize=(8,3.5))
    plt.plot(g.index, g.values, marker='o')
    plt.title(f"Média anual — {col}")
    plt.xlabel("Ano"); plt.ylabel(col)
    plt.tight_layout()
    plt.savefig(outdir / "anual.png", dpi=150)
    plt.close()


## Gerando figuras

In [15]:
for c in cols_to_interp:
    col_dir = _col_dir(FIG_DIR, c)
    plot_series_with_roll(df_clean, c, col_dir)
    plot_distributions(df_clean, c, col_dir)
    monthly_profile(df_clean, c, col_dir)
    yearly_profile(df_clean, c, col_dir)

print("Figuras salvas por coluna em subpastas de:", FIG_DIR)

Figuras salvas por coluna em subpastas de: resultados_estacao_TIJUCA\figuras


## Salvando tratamento por hora

In [16]:
project_root = Path().resolve().parents[2]    

output_dir = project_root / "data" / "DataRio" / "Estacoes_Tratadas_Por_Hora"
output_dir.mkdir(parents=True, exist_ok=True)

output_csv_path = output_dir / f"ESTACAO_{ESTACAO}_POR_HORA.csv"
df_clean.to_csv(output_csv_path, index=False, encoding='utf-8')

print(f"Arquivo salvo em: {output_csv_path}")

Arquivo salvo em: C:\Users\jhter\OneDrive - cefet-rj.br\25_2-QualiAr\data\DataRio\Estacoes_Tratadas_Por_Hora\ESTACAO_TIJUCA_POR_HORA.csv


## Criando nova feature (AQI)

https://jeap.rio.rj.gov.br/je-metinfosmac/boletim

Como calcular o AQI
https://airly.org/en/air-quality-index-caqi-and-aqi-methods-of-calculation/

| MP₁₀ (µg/m³) 24h | MP₂.₅ (µg/m³) 24h | O₃ (µg/m³) 8h | CO (ppm) 8h | NO₂ (µg/m³) 1h | SO₂ (µg/m³) 24h | Índice | Qualidade do Ar | Efeitos |
|------------------|------------------|---------------|-------------|----------------|------------------|--------|------------------|---------|
| 0 - 50           | 0 - 25           | 0 - 100       | 0 - 9       | 0 - 200        | 0 - 20           | 0 - 40 | N1 - Boa         | - |
| >50 - 100        | >25 - 50         | >100 - 130    | >9 - 11     | >200 - 240     | >20 - 40         | 41 - 80 | N2 - Moderada     | Pessoas de grupos sensíveis (crianças, idosos e pessoas com doenças respiratórias e cardíacas) podem apresentar sintomas como tosse seca e cansaço. A população em geral não é afetada. |
| >100 - 150       | >50 - 75         | >130 - 160    | >11 - 13    | >240 - 320     | >40 - 365        | 81 - 120 | N3 - Ruim         | Toda a população pode apresentar sintomas como tosse seca, cansaço, ardor nos olhos, nariz e garganta. Pessoas de grupos sensíveis (crianças, idosos e pessoas com doenças respiratórias e cardíacas) podem apresentar efeitos mais sérios na saúde. |
| >150 - 250       | >75 - 125        | >160 - 200    | >13 - 15    | >320 - 1130    | >365 - 800       | 121 - 200 | N4 - Muito Ruim   | Toda a população pode apresentar agravamento dos sintomas como tosse seca, cansaço, ardor nos olhos, nariz e garganta e ainda falta de ar e respiração ofegante. Efeitos ainda mais graves à saúde de grupos sensíveis (crianças, idosos e pessoas com doenças respiratórias e cardíacas). |
| >250 - 600       | >125 - 300       | >200 - 800    | >15 - 50    | >1130 - 3750   | >800 - 2620      | 201 - 400 | N5 - Péssima      | Toda a população pode apresentar sérios riscos de manifestações de doenças respiratórias e cardiovasculares. Aumento de mortes prematuras em pessoas de grupos sensíveis. |

### Funções auxiliares

In [17]:
BREAKPOINTS = {
    "pm10_24h": [
        (0, 50, 0, 40),
        (50, 100, 41, 80),
        (100, 150, 81, 120),
        (150, 250, 121, 200),
        (250, 600, 201, 400),
    ],
    "pm2_5_24h": [
        (0, 25, 0, 40),
        (25, 50, 41, 80),
        (50, 75, 81, 120),
        (75, 125, 121, 200),
        (125, 300, 201, 400),
    ],
    "o3_8h": [
        (0, 100, 0, 40),
        (100, 130, 41, 80),
        (130, 160, 81, 120),
        (160, 200, 121, 200),
        (200, 800, 201, 400),
    ],
    "co_8h": [
        (0, 9, 0, 40),
        (9, 11, 41, 80),
        (11, 13, 81, 120),
        (13, 15, 121, 200),
        (15, 50, 201, 400),
    ],
    "no2_1h": [
        (0, 200, 0, 40),
        (200, 240, 41, 80),
        (240, 320, 81, 120),
        (320, 1130, 121, 200),
        (1130, 3750, 201, 400),
    ],
    "so2_24h": [
        (0, 20, 0, 40),
        (20, 40, 41, 80),
        (40, 365, 81, 120),
        (365, 800, 121, 200),
        (800, 2620, 201, 400),
    ],
}

In [18]:
# Rotulagem do índice
def qual_label(idx):
    if pd.isna(idx):
        return np.nan, np.nan
    if 0 <= idx <= 40:
        return "N1 - Boa", "-"
    if 41 <= idx <= 80:
        return "N2 - Moderada", "Grupos sensíveis podem apresentar sintomas; população geral não afetada."
    if 81 <= idx <= 120:
        return "N3 - Ruim", "Sintomas em toda a população; grupos sensíveis podem ter efeitos mais sérios."
    if 121 <= idx <= 200:
        return "N4 - Muito Ruim", "Agravamento de sintomas na população; efeitos mais graves em grupos sensíveis."
    if 201 <= idx <= 400:
        return "N5 - Péssima", "Riscos sérios à saúde; aumento de mortes prematuras em grupos sensíveis."
    # Fora da escala
    return np.nan, np.nan

In [19]:
# Cálculo do subíndice por interpolação linear dentro da faixa
def calc_subindex(metric_name, concentration):
    if pd.isna(concentration):
        return np.nan
    for (c_lo, c_hi, i_lo, i_hi) in BREAKPOINTS[metric_name]:
        # inclui limite inferior, exclui superior (última faixa inclui ambos)
        if (concentration >= c_lo) and (concentration < c_hi or (concentration == c_hi and (c_hi == BREAKPOINTS[metric_name][-1][0] or (c_hi, i_hi) == BREAKPOINTS[metric_name][-1][0:2]))):
            # interpolação linear
            if c_hi == c_lo: 
                return float(i_hi)
            return float(i_lo + (i_hi - i_lo) * (concentration - c_lo) / (c_hi - c_lo))
    # Acima do último breakpoint: extrapola linearmente na última faixa
    c_lo, c_hi, i_lo, i_hi = BREAKPOINTS[metric_name][-1]
    if concentration > c_hi:
        return float(i_lo + (i_hi - i_lo) * (concentration - c_lo) / (c_hi - c_lo))
    return np.nan

### Leitura e preparo

In [20]:
df = df_clean.copy()
df["data"] = pd.to_datetime(df["data"], errors="coerce", infer_datetime_format=True)
df = df.sort_values("data").set_index("data")

# Garantir nomes esperados
colmap = {
    "pm10": "pm10",
    "pm2_5": "pm2_5",
    "o3": "o3",
    "co": "co",
    "no2": "no2",
    "so2": "so2",
}
for c in colmap.values():
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        
display(df.head())

C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\3756843583.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["data"] = pd.to_datetime(df["data"], errors="coerce", infer_datetime_format=True)


,nome_estacao,codnum,ano,mes,dia,data_dia,chuva,temp,ur,co,no,no2,nox,so2,o3,pm10,pm2_5,lat,lon
data,,,,,,,,,,,,,,,,,,,
2012-01-01 00:30:00,ESTAÇÃO TIJUCA,8,2012,1,1,2012-01-01,0.0,24.62,88.31,0.07,NaN,NaN,NaN,0.36,24.71,27.0,NaN,-22.924915,-43.232657
2012-01-01 01:30:00,ESTAÇÃO TIJUCA,8,2012,1,1,2012-01-01,0.4,23.50,93.05,0.11,NaN,NaN,NaN,0.28,22.74,28.0,NaN,-22.924915,-43.232657
2012-01-01 02:30:00,ESTAÇÃO TIJUCA,8,2012,1,1,2012-01-01,1.0,22.51,98.06,0.10,NaN,NaN,NaN,0.27,19.22,28.0,NaN,-22.924915,-43.232657
2012-01-01 03:30:00,ESTAÇÃO TIJUCA,8,2012,1,1,2012-01-01,0.0,21.47,98.40,0.06,NaN,NaN,NaN,1.51,24.55,36.0,NaN,-22.924915,-43.232657
2012-01-01 04:30:00,ESTAÇÃO TIJUCA,8,2012,1,1,2012-01-01,0.2,21.36,98.41,0.08,NaN,NaN,NaN,1.89,19.00,40.0,NaN,-22.924915,-43.232657


### Cálculo das métricas diárias
Regras:
- PM10 24h: média diária (requer cobertura mínima de horas)
- PM2.5 24h: média diária
- SO2 24h: média diária
- O3 8h: maior média móvel de 8h dentro do dia
- CO 8h: maior média móvel de 8h dentro do dia
- NO2 1h: máximo horário do dia

In [21]:
MIN_HORAS_24H = 18   # cobertura mínima para considerar o dia em médias 24h
MIN_HORAS_8H = 6     # cobertura mínima dentro da janela de 8h

# 24h means with coverage
def daily_mean_with_min(series, min_hours=MIN_HORAS_24H):
    grp = series.resample("D").agg(["mean", "count"])
    out = grp["mean"].where(grp["count"] >= min_hours)
    return out

pm10_24h = daily_mean_with_min(df["pm10"]) if "pm10" in df.columns else pd.Series(dtype=float)
pm2_5_24h = daily_mean_with_min(df["pm2_5"]) if "pm2_5" in df.columns else pd.Series(dtype=float)
so2_24h = daily_mean_with_min(df["so2"]) if "so2" in df.columns else pd.Series(dtype=float)

# 8h rolling max within day
def daily_8h_max(series, min_hours=MIN_HORAS_8H):
    if series.name is None:
        series = series.copy()
        series.name = "x"
    roll = series.rolling("8H", min_periods=min_hours).mean()
    return roll.resample("D").max()

o3_8h_max = daily_8h_max(df["o3"]) if "o3" in df.columns else pd.Series(dtype=float)
co_8h_max = daily_8h_max(df["co"]) if "co" in df.columns else pd.Series(dtype=float)

# NO2 1h max per day
no2_1h_max = df["no2"].resample("D").max() if "no2" in df.columns else pd.Series(dtype=float)

# ===== SUBÍNDICES POR POLUENTE =====
out = pd.DataFrame(index=df.resample("D").size().index)
out.index.name = "data_dia"

# Guardar métricas
if not pm10_24h.empty:  out["pm10_24h"] = pm10_24h
if not pm2_5_24h.empty: out["pm2_5_24h"] = pm2_5_24h
if not so2_24h.empty:   out["so2_24h"] = so2_24h
if not o3_8h_max.empty: out["o3_8h"] = o3_8h_max
if not co_8h_max.empty: out["co_8h"] = co_8h_max
if not no2_1h_max.empty:out["no2_1h"] = no2_1h_max

# Subíndices (interpolados nas faixas)
if "pm10_24h" in out:
    out["idx_pm10"] = out["pm10_24h"].apply(lambda v: calc_subindex("pm10_24h", v))
if "pm2_5_24h" in out:
    out["idx_pm2_5"] = out["pm2_5_24h"].apply(lambda v: calc_subindex("pm2_5_24h", v))
if "o3_8h" in out:
    out["idx_o3"] = out["o3_8h"].apply(lambda v: calc_subindex("o3_8h", v))
if "co_8h" in out:
    out["idx_co"] = out["co_8h"].apply(lambda v: calc_subindex("co_8h", v))
if "no2_1h" in out:
    out["idx_no2"] = out["no2_1h"].apply(lambda v: calc_subindex("no2_1h", v))
if "so2_24h" in out:
    out["idx_so2"] = out["so2_24h"].apply(lambda v: calc_subindex("so2_24h", v))

# ===== AQI por dia (pior subíndice do dia) =====
idx_cols = [c for c in out.columns if c.startswith("idx_")]
out["AQI"] = out[idx_cols].max(axis=1, skipna=True).round(0).astype("Int64")

# Poluente dominante (o de maior subíndice)
def dominante_row(row):
    vals = row[idx_cols]
    if vals.dropna().empty:
        return np.nan
    pol = vals.idxmax() # pega o nome do poluente com maior subíndice
    return pol.replace("idx_", "")

out["dominante"] = out.apply(dominante_row, axis=1)

# Rotulagem de qualidade e efeitos
labels = out["AQI"].apply(qual_label)
out["Qualidade_do_Ar"] = labels.apply(lambda x: x[0])
out["Efeitos"] = labels.apply(lambda x: x[1])

ordered = ["AQI", "Qualidade_do_Ar"] # "Efeitos", "dominante"
metric_cols = [c for c in ["pm10_24h","pm2_5_24h","o3_8h","co_8h","no2_1h","so2_24h"] if c in out.columns]
# ordered += metric_cols + idx_cols
out = out[ordered]

C:\Users\jhter\AppData\Local\Temp\ipykernel_27708\2103729125.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  roll = series.rolling("8H", min_periods=min_hours).mean()


In [22]:
output_dir = project_root / "data" / "DataRio" / "AQI"
output_dir.mkdir(parents=True, exist_ok=True)

output_csv_path = output_dir / f"AQI_{ESTACAO}.csv"

out.reset_index().to_csv(output_csv_path, index=False, encoding="utf-8")
print(f"AQI diário salvo em: {output_csv_path}")

AQI diário salvo em: C:\Users\jhter\OneDrive - cefet-rj.br\25_2-QualiAr\data\DataRio\AQI\AQI_TIJUCA.csv


## Gerando por dia

In [23]:
numeric_cols = df_clean.select_dtypes(include="number").columns.tolist()

# Separar chuva das demais
rain_cols = [c for c in numeric_cols if "chuva" in c.lower()]
other_numeric = [c for c in numeric_cols if c not in rain_cols]

# Agrupamento
daily_df = (
    df_clean.groupby("data_dia")
    .agg({
        "nome_estacao": "first",
        "codnum": "first",
        "ano": "first",
        "mes": "first",
        "dia": "first",
        **{col: "sum" for col in rain_cols},      # chuva = soma
        **{col: "mean" for col in other_numeric}, # demais numéricas = média
        "lat": "first",
        "lon": "first"
    })
    .reset_index()
)

# Arredondar colunas numéricas para 3 casas decimais
for col in numeric_cols:
    if col in daily_df.columns:
        daily_df[col] = daily_df[col].round(3)

daily_df.head()


,data_dia,nome_estacao,codnum,ano,mes,dia,chuva,temp,ur,co,no,no2,nox,so2,o3,pm10,pm2_5,lat,lon
0,2012-01-01,ESTAÇÃO TIJUCA,8.0,2012.0,1.0,1.0,5.2,23.226,95.108,0.124,NaN,NaN,NaN,3.373,14.330,22.375,NaN,-22.925,-43.233
1,2012-01-02,ESTAÇÃO TIJUCA,8.0,2012.0,1.0,2.0,19.0,20.066,98.163,0.264,NaN,NaN,NaN,2.998,10.927,15.708,NaN,-22.925,-43.233
2,2012-01-03,ESTAÇÃO TIJUCA,8.0,2012.0,1.0,3.0,0.0,23.855,77.530,0.162,NaN,NaN,NaN,5.776,9.933,26.208,NaN,-22.925,-43.233
3,2012-01-04,ESTAÇÃO TIJUCA,8.0,2012.0,1.0,4.0,0.0,25.741,71.502,0.171,NaN,NaN,NaN,2.354,19.879,27.458,NaN,-22.925,-43.233
4,2012-01-05,ESTAÇÃO TIJUCA,8.0,2012.0,1.0,5.0,0.0,25.865,77.911,0.103,NaN,NaN,NaN,1.582,35.781,29.167,NaN,-22.925,-43.233


In [24]:
aqi = pd.read_csv('https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataRio/AQI/AQI_TIJUCA.csv')

aqi["data_dia"] = pd.to_datetime(aqi["data_dia"]).dt.date

daily_df = daily_df.merge(aqi, on="data_dia", how="left")

int_cols = ["codnum", "ano", "mes", "dia", "AQI"]

for col in int_cols:
    if col in daily_df.columns:
        daily_df[col] = daily_df[col].astype("Int64") 

HTTPError: HTTP Error 404: Not Found

In [ ]:
display(daily_df.shape)

(4749, 23)

In [ ]:
project_root = Path().resolve().parents[2]    

output_dir = project_root / "data" / "DataRio" / "Estacoes_Tratadas_Por_Dia"
output_dir.mkdir(parents=True, exist_ok=True)

output_csv_path = output_dir / f"ESTACAO_{ESTACAO}_POR_DIA.csv"
daily_df.to_csv(output_csv_path, index=False, encoding='utf-8')

print(f"Arquivo salvo em: {output_csv_path}")

Arquivo salvo em: C:\Users\jhter\OneDrive - cefet-rj.br\qualiar\data\DataRio\Estacoes_Tratadas_Por_Dia\ESTACAO_TIJUCA_POR_DIA.csv
